<h1 style="text-align:center;">Proyecto final </h1>
<h1 style="text-align:center;">$LIMPIEZA \ DATA \ RIPLEY$</h1>



Importamos las librerías

In [41]:
import pandas as pd
import numpy as np
import re

Leemos el archivo CSV de Ripley



In [2]:
df = pd.read_csv("dataRIPLEY.csv",sep=";")

Eliminamos las columnas que no evaluaremos

In [3]:
df = df.drop(["alto (cm)", "ancho (cm)", "bluetooth", "camara posterior rango", "cantidad de cámaras", "capacidad de bateria", "chip de compañia incluido","compañia", "contrato", "enfoque automatico", "entrada de audio","flash", "formatos video", "largo (cm)","lector de huella digital", "localizacion", "nfc", "peso (kg)", "radio", "reproduce formato(s) de audio", "zona wi-fi","zoom digital (x)","procesador","sistema operativo rango","usb","tamaño de pantalla rango", "tipo de conector","tipo de producto"  ], axis=1)

Renombramos las columnas acorde al diccionario




In [4]:
df.rename(columns={
    "bateria": "bat",
    "marca": "brand",
    "camara frontal": "cam_front",
    "camara posterior": "cam_main",
    "conectividad": "conect",
    "tipo procesador": "cores",
    "velocidad cpu": "cpu_vel",
    "memoria interna": "mem_int",
    "memoria ram": "mem_ram",
    "modelo": "model",
    "sistema operativo": "os",
    "nombre": "phone",      
    "resolución de la pantalla": "screen_res",
    "tamaño de la pantalla": "screen_size",
    "tipo de pantalla": "screen_type",
    "sim": "sim",
    "garantia": "warranty",
    "resistente al agua": "water_res",
    "precio normal": "price_norm",
    "precio oferta": "price_off",
   }, inplace=True)

Agregamos las dos columnas nuevas, las cuales la data de Ripley no contaba originalmente

In [5]:
df["color"] = ""
df["condition"] = "";

Ordenamos las columnas alfabeticamente

In [6]:
df = df.reindex(sorted(df.columns), axis=1)

Añadimos información a la columna "color", para ello extraeremos la última palabra de la columna "phone" en la cual se encuentra el color de los equipos dentro del nombre de los equipos

---



In [7]:
df["color"] = df["phone"].str.rsplit(n=1, expand=True)[1]

Agregamos el datos de la condición en que se encuentran los equipos, que en este caso es "nuevo" para todos

In [8]:
df["condition"] = "nuevo"

Eliminar si encuentra un "m" o un espacio " "a partir de allí hasta el final u así obtendremos únicamente el valor numérico de la batería, luego lo transformaremos a dato flotante

In [9]:
df['bat'] = df['bat'].str.split("m").str[0].str.split(" ").str[0]

In [10]:
df["bat"] = df["bat"].astype(float)

Eliminamos la palabra "año" del tiempo de garantía, ubicado en la columna "warranty", habían 3 vacíos que daban "n", los cuales se reemplazaron por "0" y finalmente lo transformamos a flotante

In [11]:
df["warranty"] = df["warranty"].astype(str)

In [12]:
df["warranty"] = df["warranty"].apply(lambda x: x[:x.rfind(x.rstrip("0123456789"))+1])

In [13]:
df["warranty"].replace("n", "0", inplace=True)

In [14]:
df['warranty'] = df['warranty'].astype(float)

Eliminamos los símbolos de "S/" y otros como "," y " '' "

In [15]:
df["price_norm"] = df["price_norm"].astype(str).apply(lambda x: x.replace("S/",""))


In [16]:
df["price_norm"] = df["price_norm"].apply(lambda x: x.replace(",",""))

In [17]:
df["price_norm"] = df["price_norm"].apply(lambda x: x.replace("''",""))

Eliminamos desde el valor de pulgadas de los equipos en adelante de la columna "phone"

In [18]:
df["phone"] = df["phone"].str.split('"').str[0]

In [19]:
df["phone"] = df["phone"].str.split("''").str[0]

Ahora eliminamos los cuatro últimos digitos de la columna phone, en la cual se indican datos como la velocidad del procesador, a fin de continuar limpiando el nombre del modelo del celular

In [20]:
df["phone"] = df["phone"].str.slice(stop=-4)

In [21]:
df["phone"] = df["phone"].str.replace("CELULAR ", "").str.replace("APPLE ", "").str.replace("SAMSUNG", "").str.replace("HUAWEI", "")

Encontramos los datos que en lugar de decir si son resistente al agua, se le colocó otro dato y en la siguiente función se reemplaza ese valor por "si"

In [22]:
mask = df['water_res'].isin(['1 ATM', 'IP68', 'IP67', 'IPX7'])

In [23]:
df.loc[mask, 'water_res'] = 'si'

De la columna tamaño de pantalla, encontramos la unidad la cual está en pulgadas y la eliminamos, habían otros datos con otros símbolos para lo cual se aplica nuevanente replace. Finalmente, se convierte a flotante.

In [24]:
df['screen_size'] = df['screen_size'].str.replace('"', '')

In [25]:
df['screen_size'] = df['screen_size'].str.replace("''", "")

In [26]:
df['screen_size'] = df['screen_size'].astype(float)

In [27]:
#pd.set_option("display.max.rows",None)

Eliminamos las unidades de la resolución de las cámaras principal y frontal

In [28]:
df["cam_front"] = df["cam_front"].str.replace("MP", "")
df["cam_main"] = df["cam_main"].str.replace("MP", "")

In [29]:
df["cam_front"] = df["cam_front"].str.replace("Mpx", "")

In [30]:
df["cam_main"] = df["cam_main"].str.replace("MP", "")

Eliminamos también otros caracteres que no corresponden a las columnas de las cámaras frontal y principal

In [31]:
df["cam_front"] = df["cam_front"].str.replace(" ", "")
df["cam_main"] = df["cam_main"].str.replace(" ", "")

In [32]:
df["cam_main"] = df["cam_main"].str.split("+", expand=True)[0]

In [33]:
df["cam_main"] = df["cam_main"].str.replace("f/1.79", "")

<ipython-input-33-8b28ad28b3f1>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["cam_main"] = df["cam_main"].str.replace("f/1.79", "")


In [34]:
df["cam_main"] = df["cam_main"].str.replace("x", "")

Ubicamos si hay información que no corresponde en la columna color y eliminamos por ejemplo "MP" y luego transformamos a nan esos datos

In [39]:
mask = df["color"].str.contains("MP")

In [42]:
df.loc[mask, "color"] = np.nan

Revisamos la columna "conect" y eliminamos los espacios vacíos y las palabras que no corresponden.

In [43]:
df["conect"] = df["conect"].str.replace(" ", "")

In [44]:
df['conect'] = df['conect'].astype(str)

In [45]:
df["conect"] = df["conect"].str.replace("Bluetooth", "")

In [46]:
df["conect"] = df["conect"].str.replace("Wi-Fi", "").replace("GPS", "").replace("NFC", "").replace("USB", "").replace("tipoC", "").replace("LTE", "").replace("BT", "").replace("Wifi", "").replace("WLAN", "").replace(",", "").replace("3G/4G/", "").replace("/4G/", "").replace("3G2G", "").replace("/", "").replace(".0","G")


Eliminamos las unidades como "GB" de las columnas de memoria interna y memoria ram. Luego convertimos a string la columna velocidad del CPU.

In [47]:
df["mem_int"] = df["mem_int"].str.replace("GB","")

In [48]:
df["mem_ram"] = df["mem_ram"].str.replace("GB","")

In [49]:
df["cpu_vel"] = df["cpu_vel"].astype(str)

Elminamos todo lo que está después de símbolo "GHz" para dejar únicamente el valor numérico del campo de la columna velocidad del CPU

In [50]:
def extract_number(string):
    match = re.findall("\d+\.\d+", string)
    return float(match[0]) if match else None

df['cpu_vel'] = df['cpu_vel'].str.split('GHz').str[0].apply(extract_number)

Ordenamos las columnas, de tal manera que al final queden las columnas precio

In [51]:
dff=df[['bat','brand','cam_front','cam_main','color','condition','conect','cores','cpu_vel','mem_int','mem_ram','model','os','phone','screen_res','screen_size','screen_type','sim','warranty','water_res','price_norm','price_off']]


In [52]:
pd.set_option("display.max.rows",None)

In [ ]:
#dff.to_csv("dataRIPLEY_final.csv", index=False)